In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import *
from IPython.display import display, HTML, IFrame
%matplotlib inline
from collections import defaultdict
from IPython.display import set_matplotlib_formats
from bs4 import BeautifulSoup
set_matplotlib_formats('png','pdf')


In [2]:
folder = ['compsci-data','jbt-data', 'brands-data-400', 'brands-data-100', 'brands-data-it2', 'jbt-data-it2', 'compsci-data-it2']
MERGE_OTHER = True
ut = 'jbt'
#folder = ['brands-data-it2', 'brands-data-it3', 'brands-data-400', 'brands-data-100']
folder = ['{}-data'.format(ut), '{}-data-it2'.format(ut)]

In [3]:


agg_list = []
full_list = []
test_list = []
for f in folder:
    agg =  pd.read_csv('{}/agg.csv'.format(f))
    agg['type'] = f.split('-')[0]
    agg.set_index('id')
    agg['file'] = f
    agg_list.append(agg)
   
    full = pd.read_csv('{}/full.csv'.format(f))
    full['type'] = f.split('-')[0]
    full.set_index('id')
    full_list.append( full)
    test = pd.read_csv('{}/test.csv'.format(f))

    test = test[test['_hidden'] == False]
    print('{} {}'.format(f,len(test)))
    test['type'] = f.split('-')[0]
    test.set_index('id')
    
    test_list.append (test)
aggregated = pd.concat(agg_list)
if MERGE_OTHER:
    aggregated['label'] = aggregated.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)
    full['label'] = full.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)


print(aggregated.describe())
full = pd.concat(full_list)
test_questions = pd.concat(test_list)

aggregated = aggregated[aggregated['_unit_state'] == 'finalized']
aggregated['raw_text'] = aggregated.apply(
        lambda row: BeautifulSoup(row['text_html'], "lxml").text.replace(':[OBJECT_A]', '').replace(':[OBJECT_B]', ''),
        axis=1)


def print_info(data):
    avg = pd.merge(data, full, suffixes=['_l', '_r'], on=['id'], how='left')[['id', 'text_readable_r', 'label:confidence', 'label_l', '_golden', 'label_r', '_trust','a','b']].sort_values(['label:confidence', 'label_l' ,'id'])
    renamed_avg = avg.rename(columns={'label_l' : 'assigned_by_crowdflower', 'label_r' : 'proposed_by_annotator'})
    renamed_avg
    return renamed_avg


jbt-data 51
jbt-data-it2 51
           _unit_id  _trusted_judgments  label:confidence  label_gold
count  2.463000e+03         2463.000000       2463.000000         0.0
mean   1.566124e+09            3.051563          0.897735         NaN
std    1.511396e+07            0.221188          0.171038         NaN
min    1.543287e+09            3.000000          0.333300         NaN
25%    1.543287e+09            3.000000          0.694450         NaN
50%    1.576122e+09            3.000000          1.000000         NaN
75%    1.576123e+09            3.000000          1.000000         NaN
max    1.576124e+09            4.000000          1.000000         NaN


In [4]:
from sklearn.model_selection import train_test_split

train_i, test_i = train_test_split(aggregated,stratify=aggregated['label'], test_size=0.2)
print('{} {}'.format(len(train_i), len(test_i)))
print(train_i['_golden'].unique())
print(test_i['_golden'].unique())

1970 493
[False]
[False]


In [5]:
to_remove = test_questions['id'].unique()
print(aggregated.columns)
aggregated = aggregated[~aggregated['id'].isin( to_remove)][['id', 'a', 'b', 'raw_text', 'text_html', 'text_readable', 'type', 'marker','file']]
aggregated['label_gold'] = ''
aggregated['label_gold_reason'] = ''
aggregated['_golden'] = False
aggregated.to_csv('{}-sentences.csv'.format(ut), index=False)
len(aggregated['id'].unique())

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'label', 'label:confidence', 'a',
       'also_acceptable', 'b', 'gold_label', 'id', 'label_gold', 'marker',
       'raw_text', 'text_html', 'text_readable', 'type', 'file'],
      dtype='object')


2439

In [6]:
testq = test_questions.drop_duplicates(subset=['id'])[['id', 'label_gold', 'text_html', 'text_readable', 'marker', 'a', 'b', 'raw_text', 'type', 'label_gold_reason']]
testq['label_gold'] = testq.apply(lambda row: testq['label_gold'].str.replace('OTHER\n', ''), axis=0)
testq['_golden'] = True
testq.to_csv('{}-test-questions.csv'.format(ut), index=False)
testq

,id,label_gold,text_html,text_readable,marker,a,b,raw_text,type,label_gold_reason,_golden
0,P2533877672,OTHER,"The Jimmy Dean <span style=""color: #9A14B2; fo...",The Jimmy Dean *sandwich* had far better §saus...,True,sandwich,sausage,The Jimmy Dean sandwich had far better sausage.,jbt,"The sentence is comparative, but it does not c...",True
1,E559271740,NONE,"Decent <span style=""color: #9A14B2; font-weigh...",Decent *pasta* and excellent §pizza$ made in a...,True,pizza,pasta,Decent pasta and excellent pizza made in a woo...,jbt,"The sentence is not comparative. ""Decent"" and ...",True
2,F805035826,NONE,"<span style=""color: #9A14B2; font-weight: bold...","*georgia* TECH 56, §virginia$ 20 : The Cavalie...",True,virginia,georgia,"GEORGIA TECH 56, VIRGINIA 20 : The Cavaliers f...",jbt,NaN,True
3,L1806350316,BETTER,"<span style=""color: #9A14B2; font-weight: bold...","*nixon* was better in the second, gerald ford ...",True,reagan,nixon,"nixon was better in the second, gerald ford ca...",jbt,NaN,True
4,N2239679026,NONE,"So, is <span style=""color: #9A14B2; font-weigh...","So, is *google* Plus better than §facebook$?",True,google,facebook,"So, is Google Plus better than Facebook?",jbt,The sentence is a question. As written in the ...,True
5,H1097617435,BETTER,"If there's anything worse than a <span style=""...",If there's anything worse than a *harvard* man...,True,harvard,princeton,"If there's anything worse than a Harvard man, ...",jbt,The sentence is comparative and it compares Ha...,True
6,Q2714104352,NONE,"The size though, allows it to have an in-house...","The size though, allows it to have an in-house...",True,restaurant,cafe,"The size though, allows it to have an in-house...",jbt,"The sentence does not compare ""Cafe"" and ""Rest...",True
7,E653701837,NONE,"Enjoy stations such as <span style=""color: #9A...","Enjoy stations such as *fox* 101, §espn$ 1380 ...",True,fox,espn,"Enjoy stations such as Fox 101, ESPN 1380 - WY...",jbt,The sentence is not comparative.,True
8,I1350818111,NONE,"Is Netflix better than <span style=""color: #9A...",Is Netflix better than *hulu* Plus or §amazon$...,True,amazon,hulu,Is Netflix better than Hulu Plus or Amazon Prime?,jbt,The sentence is a question. As written in the ...,True
9,R2922605663,NONE,To make things worse Joy has scheduled the <sp...,To make things worse Joy has scheduled the *we...,True,wedding,birthday,To make things worse Joy has scheduled the wed...,jbt,The sentence is not comparative. NONE is the c...,True


### Quality

Sentences with a confidence <= 0.6. Either all three annotators gave a different answer or the annotators are not trustworthy enough.

In [7]:
import re
from collections import defaultdict
from pprint import pprint
import operator
d = defaultdict(int)
exp = r'[\w\s+*-]+(?=:\[OBJECT_[AB]\])'
s = set()
for i, row in aggregated.iterrows():
    a = (sorted(re.findall(exp, row['text_html'])))
    d['{} vs. {}'.format(a[0],a[1])] += 1
print(len(d))
pprint(sorted(d.items(), key=operator.itemgetter(1)))

167
[('foam vs. rubber', 2),
 ('foam vs. wood', 4),
 ('concrete vs. steel', 5),
 ('lumber vs. timber', 6),
 ('hockey vs. tennis', 6),
 ('concrete vs. rubber', 6),
 ('concrete vs. wood', 6),
 ('potato vs. steak', 6),
 ('metal vs. plywood', 8),
 ('ds vs. ps3', 9),
 ('milk vs. tea', 9),
 ('chocolate vs. tea', 9),
 ('basketball vs. hockey', 9),
 ('football vs. tennis', 9),
 ('hockey vs. lacrosse', 9),
 ('forest vs. timber', 9),
 ('beer vs. champagne', 9),
 ('cardboard vs. plastic', 9),
 ('pasta vs. pizza', 10),
 ('georgia vs. michigan', 10),
 ('skiing vs. swimming', 10),
 ('ps2 vs. wii', 10),
 ('concrete vs. timber', 10),
 ('beer vs. milk', 10),
 ('motorcycle vs. truck', 10),
 ('cardboard vs. wood', 10),
 ('birthday vs. holiday', 10),
 ('kennedy vs. nixon', 10),
 ('bread vs. soup', 10),
 ('fishing vs. skiing', 10),
 ('pistol vs. rifle', 10),
 ('steel vs. timber', 10),
 ('mobile vs. wallet', 11),
 ('concrete vs. plastic', 11),
 ('playstation vs. wii', 11),
 ('paint vs. pen', 11),
 ('pasta v

## Results

**Label Distribution**

In [8]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

def show_dist(labels):

    plt.figure(figsize=plt.figaspect(1))
    plt.pie(
        labels.value_counts().values,
        labels=labels.value_counts().keys().tolist(),
        radius=2,
        autopct=make_autopct(labels.value_counts().values))
    plt.show()


**All sentences**

In [13]:
compsci = pd.read_csv('compsci-sentences.csv')
brands = pd.read_csv('brands-sentences.csv')
jbt = pd.read_csv('jbt-sentences.csv')
combi = pd.concat([compsci,brands,jbt])
print('{}+{}+{}={} ({})'.format(len(compsci), len(brands),len(jbt),len(combi), (len(compsci)+len(brands)+len(jbt))))
jbt.duplicated('id').unique()

2425+2522+2439=7386 (7386)


array([False])

In [10]:
test_questions[~test_questions.duplicated('id')]

,_id,_pct_missed,_judgments,_hidden,_contention,_pct_contested,_gold_pool,_review_state,_label,label_gold,...,id,text_html,text_readable,gold_label,a,b,marker,raw_text,also_acceptable,type
0,1543289974,0.90,51,False,no comparison\nhad far better - comparison\nit...,0.0588,NaN,passed,NaN,OTHER,...,P2533877672,"The Jimmy Dean <span style=""color: #9A14B2; fo...",The Jimmy Dean *sandwich* had far better §saus...,O,sandwich,sausage,True,The Jimmy Dean sandwich had far better sausage.,N,jbt
1,1543290673,0.40,40,False,NaN,0.0000,NaN,NaN,NaN,NONE,...,E559271740,"Decent <span style=""color: #9A14B2; font-weigh...",Decent *pasta* and excellent §pizza$ made in a...,NaN,pizza,pasta,True,Decent pasta and excellent pizza made in a woo...,NaN,jbt
2,1543291256,0.51,43,False,ran into a better team,0.0465,NaN,passed,NaN,NONE,...,F805035826,"<span style=""color: #9A14B2; font-weight: bold...","*georgia* TECH 56, §virginia$ 20 : The Cavalie...",NaN,virginia,georgia,True,"GEORGIA TECH 56, VIRGINIA 20 : The Cavaliers f...",NaN,jbt
3,1543291617,0.32,41,False,NaN,0.0244,NaN,passed,BETTER\nOTHER\nWORSE\nNONE,BETTER,...,L1806350316,"<span style=""color: #9A14B2; font-weight: bold...","*nixon* was better in the second, gerald ford ...",NaN,reagan,nixon,True,"nixon was better in the second, gerald ford ca...",NaN,jbt
4,1543292000,0.65,49,False,Please review.,0.0204,NaN,passed,NaN,NONE,...,N2239679026,"So, is <span style=""color: #9A14B2; font-weigh...","So, is *google* Plus better than §facebook$?",NaN,google,facebook,True,"So, is Google Plus better than Facebook?",NaN,jbt
5,1543292760,0.12,52,False,NaN,0.0000,NaN,NaN,NaN,BETTER,...,H1097617435,"If there's anything worse than a <span style=""...",If there's anything worse than a *harvard* man...,NaN,harvard,princeton,True,"If there's anything worse than a Harvard man, ...",NaN,jbt
6,1543293271,0.03,39,False,NaN,0.0000,NaN,NaN,NaN,OTHER\nNONE,...,Q2714104352,"The size though, allows it to have an in-house...","The size though, allows it to have an in-house...",NaN,restaurant,cafe,True,"The size though, allows it to have an in-house...",NaN,jbt
7,1543293719,0.20,41,False,in instruction you are wrote - if we can chang...,0.0244,NaN,passed,NaN,NONE,...,E653701837,"Enjoy stations such as <span style=""color: #9A...","Enjoy stations such as *fox* 101, §espn$ 1380 ...",NaN,fox,espn,True,"Enjoy stations such as Fox 101, ESPN 1380 - WY...",NaN,jbt
8,1543294138,0.75,52,False,"in the sentence there is a comparison, please ...",0.0192,NaN,NaN,NaN,NONE,...,I1350818111,"Is Netflix better than <span style=""color: #9A...",Is Netflix better than *hulu* Plus or §amazon$...,NaN,amazon,hulu,True,Is Netflix better than Hulu Plus or Amazon Prime?,NaN,jbt
9,1543295981,0.33,36,False,NaN,0.0000,NaN,NaN,NaN,NONE,...,R2922605663,To make things worse Joy has scheduled the <sp...,To make things worse Joy has scheduled the *we...,N,wedding,birthday,True,To make things worse Joy has scheduled the wed...,NaN,jbt
